# LOAD DATA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer, FunctionTransformer, StandardScaler
from sklearn.linear_model import SGDRegressor, Lasso, LinearRegression, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyRegressor
from Plots import plot_quantiles, plot_linear_relation, plot_categories_mean, plot_multilabel_categories_mean
from CustomExtractors import extract_numbers, extract_rating

cast = pd.read_csv('./data/Movie Cast.csv')
movies = pd.read_csv('./data/Movie Data.csv')

print(cast.shape)
cast.head()

(203333, 4)


,Movie,Type,Name,Role
0,"10,000 B.C.",Actors,Steven Strait,D'Leh
1,"10,000 B.C.",Actors,Camilla Belle,Evolet
2,"10,000 B.C.",Actors,Cliff Curtis,Tic-Tic
3,"10,000 B.C.",Actors,Reece Ritchie,Moha
4,"10,000 B.C.",Actors,Marco Khan,One Eye


In [2]:
print(movies.shape)
movies.head()

(5691, 16)


,Movie,Budget (thousands of $),Domestic Box Office Revenue (thousands of $),International Box Office Revenue (thousands of $),MPAA Rating,Running time,Franchise,Original source,Genre,Production method,Type,Production companies,Production country,Languages,Distributor,Release year
0,10 Questions for the Dalai Lama,NaN,224.5,260.0,Not Rated,NaN,NaN,Real Life Events,Documentary,Live Action,Factual,NaN,United States,English,Monterey Media,2007
1,10th & Wolf,8000.0,54.7,89.1,Not Rated,NaN,NaN,Real Life Events,Drama,Live Action,Dramatization,NaN,United States,English,ThinkFilm,2006
2,2006 Academy Award Nominated Short Films,NaN,335.1,NaN,Not Rated,NaN,Academy Award Short Film Nominations,Compilation,Thriller,Multiple Production Methods,Multiple Creative Types,NaN,United States,English,Magnolia Pictures,2007
3,24 Hour Party People,NaN,1169.0,2435.9,"R for strong language, drug use and sexuality",NaN,NaN,Real Life Events,Drama,Live Action,Dramatization,NaN,United States,English,MGM,2002
4,39 Pounds of Love,NaN,28.1,2.1,Not Rated,NaN,NaN,Real Life Events,Documentary,Live Action,Factual,NaN,United States,English,Balcony Releasing,2005


In [3]:
# we'll change the table names to lower case and substitute spaces for underscores
# this makes it easier to type the names of the variables
movies.columns = [col_name.replace(" ", "_").lower() for col_name in movies.columns]
cast.columns = [col_name.replace(" ", "_").lower() for col_name in cast.columns]

# 1. REVENUE MODEL
## Target preparation

In [4]:
# change names of budget and revenue variables
movies.rename(columns={'budget_(thousands_of_$)': 'budget',
  'domestic_box_office_revenue_(thousands_of_$)': 'us_revenue',
  'international_box_office_revenue_(thousands_of_$)': 'international_revenue'}, inplace=True)

money_cols = [
    'international_revenue',
    'us_revenue',
    'budget'
]

movies[money_cols] = movies[money_cols].fillna(0)

# Change the scale of dollar variables to millions
movies[money_cols] = movies[money_cols] / 1000

# Target column is sum of all revenue
movies['total_revenue'] = movies['us_revenue'] + movies['international_revenue']

movies.columns

Index(['movie', 'budget', 'us_revenue', 'international_revenue', 'mpaa_rating',
       'running_time', 'franchise', 'original_source', 'genre',
       'production_method', 'type', 'production_companies',
       'production_country', 'languages', 'distributor', 'release_year',
       'total_revenue'],
      dtype='object')

In [5]:
# Filter movies without budget information and too small budgets
movies = movies[(movies.total_revenue > 0.1) & (movies.budget > 1)].reset_index(drop=True)
movies.shape

(3378, 17)

In [6]:
# Profits
y = movies['total_revenue'] - movies['budget']

X = movies.drop(['international_revenue','us_revenue','total_revenue'], axis=1)

In [7]:
# Divide data into training and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.8,
    test_size=0.2,
)

print("Train has " + str(X_train.shape[0]) + " rows")
print("Test has " + str(X_test.shape[0]) + " rows")

Train has 2702 rows
Test has 676 rows


## Variable analysis

### Numerical variables

In [8]:
def table_quantiles(feature, target, col_names, other_col=None, bins=10):
    df_dict = {'feature': feature, 'target': target, 'other': other_col} if other_col is not None else {'feature': feature, 'target': target}
    df = pd.DataFrame(df_dict)

    df['quantile'] = np.floor(bins * (df['feature'].rank(method='first') - 1) / df.shape[0])
    
    agg_dict = {'feature': 'max', 'target': 'mean', 'other': 'mean'} if other_col is not None else {'feature': 'max', 'target': 'mean'}
    df = df.groupby('quantile').agg(agg_dict).sort_values('quantile').reset_index(drop=False)
    rename_dict = {'feature': col_names[0], 'target': col_names[1], 'other': col_names[2]} if other_col is not None else {'feature': col_names[0], 'target': col_names[1]}
    
    return df.rename(columns=rename_dict)

In [9]:
## Budget
table_quantiles(X_train.budget, y_train, ['budget (max)', 'profit'])

,quantile,budget (max),profit
0,0.0,5.6,20.412058
1,1.0,10.0,24.416285
2,2.0,15.0,40.921531
3,3.0,20.0,40.985677
4,4.0,28.0,40.866672
5,5.0,37.0,50.428999
6,6.0,50.0,62.546219
7,7.0,70.0,93.264944
8,8.0,108.0,149.381459
9,9.0,400.0,400.235839


In [10]:
## Release year
table_quantiles(X_train.release_year, y_train, ['release_year (max)', 'profit', 'mean budget'], X_train.budget)

,quantile,release_year (max),profit,mean budget
0,0.0,2001,89.857282,39.987344
1,1.0,2004,69.873544,42.160750
2,2.0,2006,63.950211,45.170474
3,3.0,2008,60.939148,36.968630
4,4.0,2009,80.431186,43.430185
5,5.0,2011,73.192636,42.317851
6,6.0,2013,109.552765,49.227222
7,7.0,2015,119.013455,50.005926
8,8.0,2017,113.425091,46.407465
9,9.0,2019,142.882849,53.006520


In [11]:
# Extract running time
X_train['duration'] = X_train.running_time.str.extract('(\d+)').astype('float64')
X_train.duration = X_train.duration.fillna(X_train.duration.median())

X_test['duration'] = X_test.running_time.str.extract('(\d+)').astype('float64')
X_test.duration = X_test.duration.fillna(X_train.duration.median())

In [12]:
table_quantiles(X_train.duration, y_train, ['duration (max)', 'profit', 'mean budget'], X_train.budget)

,quantile,duration (max),profit,mean budget
0,0.0,90.0,64.480098,33.963561
1,1.0,95.0,77.269220,36.173732
2,2.0,100.0,61.180245,34.179722
3,3.0,104.0,69.814287,40.061581
4,4.0,106.0,31.447505,27.805000
5,5.0,109.0,55.282067,31.134320
6,6.0,114.0,72.717151,43.067419
7,7.0,121.0,98.996161,49.249837
8,8.0,131.0,139.903066,59.989444
9,9.0,201.0,252.188691,93.121481


In [13]:
## Profit increases with minutes? It might correlate with budget
long_movie_budget = X_train[X_train.duration > 140].agg({'budget': 'mean'})
movie_budget = X_train.agg({'budget': 'mean'})

print(f'Avg. budget of movie: {movie_budget}')
print(f'Avg. budget of movie over 140 min: {long_movie_budget}')

Avg. budget of movie: budget    44.865487
dtype: float64
Avg. budget of movie over 140 min: budget    105.537008
dtype: float64


### Categorical variables

In [14]:
def categorical_table(X, y, feat_name):
    df = pd.DataFrame({feat_name: X[feat_name], 'N': np.zeros(X.shape[0]), 'profit': y, 'budget': X.budget})
    df = df.groupby(feat_name).agg({'N':'count', 'profit':'mean', 'budget': 'mean'}).sort_values('N', ascending=False)
    print(df)
    
    return df

def one_hot_encode(X, categories, feat_name, n_other=40):
    X_return = X.copy()
    common_feats = categories.index[df.N > n_other]

    for feat in common_feats:
        X_return[f'{feat_name}_{feat}'] = (X_return[feat_name] == feat).astype(int)
    
    X_return[f'{feat_name}_Other'] = (~X_return[feat_name].isin(common_feats)).astype(int)
    
    return X_return

In [15]:
## Genre
df = categorical_table(X_train, y_train, 'genre')

                   N      profit     budget
genre                                      
Drama            683   42.372727  26.331317
Comedy           498   43.815425  27.817721
Adventure        401  215.373902  87.294763
Action           349  178.341812  86.757880
Thriller         314   60.143002  37.130202
Horror           193   72.775208  20.212176
Romantic Comedy  129   58.018670  30.289302
Musical           45  137.309419  47.951711
Dark Comedy       37   48.971824  28.674324
Documentary       23   32.169062  12.191304
Western           21   27.656905  50.571429
Performance        9   34.620237   8.555556


In [16]:
X_train = one_hot_encode(X_train, df, 'genre')
X_test = one_hot_encode(X_test, df, 'genre')